In [ ]:
import os
import pandas as pd

# set working directory
os.chdir('/home/gyk/project/lw_atac_nf')

boolean_file = 'results/bwa/merged_library/macs2/narrow_peak/consensus/consensus_peaks.mLb.clN.boolean.txt'
annotate_file = 'results/bwa/merged_library/macs2/narrow_peak/consensus/consensus_peaks.mLb.clN.annotatePeaks.txt'

# Read the boolean file and the annotate file
boolean_file_df = pd.read_csv(boolean_file, sep='\t', header = 0)
annotate_file_df = pd.read_csv(annotate_file, sep='\t', header = 0)

# Filter rows where all three replicates are True
replicate_columns_true = ['Ava_REP1.mLb.clN.bool', 
                    'Ava_REP2.mLb.clN.bool', 
                    'Ava_REP3.mLb.clN.bool']

replicate_columns_false = ['DZ_REP1.mLb.clN.bool', 
                    'DZ_REP2.mLb.clN.bool', 
                    'DZ_REP3.mLb.clN.bool']

# Select rows where all replicate_columns_true are True and all replicate_columns_false are False
filtered_boolean_file_df = boolean_file_df[(boolean_file_df[replicate_columns_true].all(axis=1))]
filtered_boolean_file_df = filtered_boolean_file_df[(filtered_boolean_file_df['DZ_REP1.mLb.clN.bool'] == False) & (filtered_boolean_file_df['DZ_REP2.mLb.clN.bool'] == False) & (filtered_boolean_file_df['DZ_REP3.mLb.clN.bool'] == False)]

print(len(filtered_boolean_file_df))

# filter the dataframes based on interval_id
interval_ids = filtered_boolean_file_df['interval_id']
filtered_annotate_df = annotate_file_df[annotate_file_df.iloc[:, 0].isin(interval_ids)]

filtered_annotate_df[["Chr", "Start", "End", "PeakID (cmd=annotatePeaks.pl consensus_peaks.mLb.clN.bed Bomo_genome_assembly_chr.fa -gid -gtf Bomo_gene_models_chr.gtf -cpu 6)",  "Nearest PromoterID","Strand",]].to_csv('analysis/results/DZ_Ava_diff_consensus_peaks.bed', sep='\t', index=False)

promoter_df = filtered_annotate_df[filtered_annotate_df['Annotation'].str.split('-').str[0] == 'promoter']
promoter_df[["Chr", "Start", "End", "PeakID (cmd=annotatePeaks.pl consensus_peaks.mLb.clN.bed Bomo_genome_assembly_chr.fa -gid -gtf Bomo_gene_models_chr.gtf -cpu 6)",  "Nearest PromoterID","Strand",]].to_csv('analysis/results/DZ_Ava_diff_consensus_peaks_promoter.bed', sep='\t', index=False, header=False)

In [ ]:
# sort the dataframes based on Chr and Start
filtered_annotate_df.sort_values(by=['Chr', 'Start'], inplace=True)
filtered_annotate_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = filtered_annotate_df

# Custom interval
bins = [-float('inf'), -100000, -10000, -5000, -3000, -1000, 0, 1000, 3000, 5000, 10000, 100000, float('inf')]
labels = [
    "L >100 kb", "L 10-100 kb", "L 5-10 kb", "L 3-5 kb", "L 1-3 kb", "L 0-1 kb", 
    "TSS", "R 0-1 kb", "R 1-3 kb", "R 3-5 kb", "R 5-10 kb", "R 10-100 kb"
]

# Dividing distances into intervals
df['Group'] = pd.cut(df['Distance to TSS'], bins=bins, labels=labels, right=False)
print(df['Group'])
# Count the number of each interval
group_counts = df['Group'].value_counts(sort=False)

# Setting CNS Style
sns.set_theme(style="whitegrid", palette="muted")

# Plotting frequency histograms
plt.figure(figsize=(8, 6))
sns.barplot(x=group_counts.index, y=group_counts.values, palette="muted")


plt.xlabel("Distance to TSS")
plt.ylabel('Count')
plt.title('Distribution of Peak Distance to TSS')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Label numbers on histograms
for i, count in enumerate(group_counts):
    ax.text(i, count + 0.1, str(count), ha='center', va='bottom', fontsize=10)

# Show chart
plt.savefig("analysis/results/promoter_distance_frequency.pdf", dpi=720, bbox_inches="tight")
plt.show()

# Show chart
plt.show()

In [ ]:
# calculate the portion of all kind of peaks on Annotation
df_annotation = filtered_annotate_df['Annotation'].str.split('(').str[0]
annotation_frequency = df_annotation.value_counts()

print(annotation_frequency)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Setting CNS Style
sns.set_theme(style="whitegrid", palette="muted")

# Plotting frequency histograms
plt.figure(figsize=(8, 6))
sns.barplot(x=annotation_frequency.index, y=annotation_frequency.values, palette="muted")

# Set Title and Label
plt.title("Annotation Frequency", fontsize=16, weight='bold')
plt.xlabel("Annotations", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Display graphics
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
# Save pictures locally
plt.savefig("analysis/results/annotation_frequency.pdf", dpi=720, bbox_inches="tight")
plt.show()


In [ ]:
peak_distance_to_tss_frequency = filtered_annotate_df['Distance to TSS'].value_counts().sort_index()
print(peak_distance_to_tss_frequency)
# Plotting frequency histograms
# Setting CNS Style
sns.set_theme(style="ticks")

# Plotting frequency histograms
plt.figure(figsize=(8, 6))
sns.jointplot(x=peak_distance_to_tss_frequency.index, y=peak_distance_to_tss_frequency.values, kind = "hex")

# Set Title and Label
# plt.title("Promoter Distance Frequency", fontsize=16, weight='bold')
plt.xlabel("Peak Distance to TSS", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Display graphics
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("analysis/results/promoter_distance_frequency.pdf", dpi=720, bbox_inches="tight")
plt.show()

In [ ]:
nearest_geneID = df_promoter["Nearest PromoterID"]
nearest_geneID.head()
nearest_geneID.to_csv("analysis/results/nearest_TSS_promoterID.csv", index=False, header=False)